In [3]:
import os

from dotenv import load_dotenv
from mem0 import Memory
from openai import OpenAI

In [4]:
load_dotenv()

True

In [5]:
client = OpenAI()

In [6]:
HOST = "localhost"

# Personalized AI Tutor

In [7]:
class PersonalAITutor:
    def __init__(self):
        """
        Initialize the PersonalAITutor with memory configuration and OpenAI client.
        """
        config = {
            "vector_store": {
                "provider": "pgvector",
                "config": {
                    "user": os.getenv("POSTGRES_USER"),
                    "password": os.getenv("POSTGRES_PASSWORD"),
                    "host": HOST,
                    "port": os.getenv("POSTGRES_LOCAL_PORT"),
                    "dbname": "mem0",
                    "embedding_model_dims": 1536,
                },
            },
        }
        self.memory = Memory.from_config(config)
        self.client = client
        self.app_id = "app-1"

    def ask(self, question, user_id=None):
        """
        Ask a question to the AI and store the relevant facts in memory

        :param question: The question to ask the AI.
        :param user_id: Optional user ID to associate with the memory.
        """
        # Start a streaming chat completion request to the AI
        stream = self.client.chat.completions.create(
            model="gpt-4o-mini",
            stream=True,
            messages=[
                {"role": "system", "content": "You are a personal AI Tutor."},
                {"role": "user", "content": question},
            ],
        )
        # Store the question in memory
        self.memory.add(question, user_id=user_id, metadata={"app_id": self.app_id})

        # Print the response from the AI in real-time
        for chunk in stream:
            if chunk.choices[0].delta.content is not None:
                print(chunk.choices[0].delta.content, end="")

    def get_memories(self, user_id=None):
        """
        Retrieve all memories associated with the given user ID.

        :param user_id: Optional user ID to filter memories.
        :return: List of memories.
        """
        return self.memory.get_all(user_id=user_id)

In [8]:
# Instantiate the PersonalAITutor
ai_tutor = PersonalAITutor()

In [9]:
# Define a user ID
user_id = "tam-1a"

In [10]:
# Ask a question
ai_tutor.ask(
    "I am learning introduction to CS. What is queue? Briefly explain.", user_id=user_id
)

A queue is a data structure that follows the First-In-First-Out (FIFO) principle. This means that the first element added to the queue will be the first one to be removed. You can think of it like a line of people waiting to buy tickets: the person who arrives first is served first.

Key operations associated with a queue include:

1. **Enqueue**: Adding an item to the end of the queue.
2. **Dequeue**: Removing an item from the front of the queue.
3. **Peek/Front**: Viewing the item at the front of the queue without removing it.
4. **IsEmpty**: Checking whether the queue is empty.

Queues are commonly used in scenarios such as scheduling tasks, managing requests in servers, and buffering data in streaming applications.

# Customer Support AI Agent

In [11]:
class CustomerSupportAIAgent:
    def __init__(self):
        """
        Initialize the CustomerSupportAIAgent with memory configuration and OpenAI client.
        """
        config = {
            "vector_store": {
                "provider": "pgvector",
                "config": {
                    "user": os.getenv("POSTGRES_USER"),
                    "password": os.getenv("POSTGRES_PASSWORD"),
                    "host": HOST,
                    "port": os.getenv("POSTGRES_LOCAL_PORT"),
                    "dbname": "mem0",
                    "embedding_model_dims": 1536,
                },
            },
        }
        self.memory = Memory.from_config(config)
        self.client = OpenAI()
        self.app_id = "customer-support"

    def handle_query(self, query, user_id=None):
        """
        Handle a customer query and store the relevant information in memory.

        :param query: The customer query to handle.
        :param user_id: Optional user ID to associate with the memory.
        """
        # Start a streaming chat completion request to the AI
        stream = self.client.chat.completions.create(
            model="gpt-4o-mini",
            stream=True,
            messages=[
                {"role": "system", "content": "You are a customer support AI agent."},
                {"role": "user", "content": query},
            ],
        )
        # Store the query in memory
        self.memory.add(query, user_id=user_id, metadata={"app_id": self.app_id})

        # Print the response from the AI in real-time
        for chunk in stream:
            if chunk.choices[0].delta.content is not None:
                print(chunk.choices[0].delta.content, end="")

    def get_memories(self, user_id=None):
        """
        Retrieve all memories associated with the given customer ID.

        :param user_id: Optional user ID to filter memories.
        :return: List of memories.
        """
        return self.memory.get_all(user_id=user_id)

In [12]:
# Instantiate the CustomerSupportAIAgent
support_agent = CustomerSupportAIAgent()

In [13]:
# Define a customer ID
customer_id = "tam-1b"

In [14]:
# Handle a customer query
support_agent.handle_query(
    "I need help with my recent order. It hasn't arrived yet.", user_id=customer_id
)

I’m sorry to hear that your order hasn’t arrived yet. Can you please provide me with your order number and any additional details such as the date you placed the order? This will help me assist you better.

# Personal AI Travel Assistant

In [15]:
config = {
    "llm": {
        "provider": "openai",
        "config": {
            "model": "gpt-4o-mini",
            "temperature": 0.1,
            "max_tokens": 2000,
        },
    },
    "embedder": {"provider": "openai", "config": {"model": "text-embedding-3-small"}},
    "vector_store": {
        "provider": "pgvector",
        "config": {
            "user": os.getenv("POSTGRES_USER"),
            "password": os.getenv("POSTGRES_PASSWORD"),
            "host": HOST,
            "port": os.getenv("POSTGRES_LOCAL_PORT"),
            "dbname": "mem0",
            "embedding_model_dims": 1536,
        },
    },
    "version": "v1.1",
}


class PersonalTravelAssistant:
    def __init__(self):
        self.client = OpenAI()
        self.memory = Memory.from_config(config)
        self.messages = [
            {"role": "system", "content": "You are a personal AI Assistant."}
        ]

    def ask_question(self, question, user_id):
        # Fetch previous related memories
        previous_memories = self.search_memories(question, user_id=user_id)
        prompt = question
        if previous_memories:
            prompt = f"User input: {question}\n Previous memories: {previous_memories}"
        self.messages.append({"role": "user", "content": prompt})

        # Generate response using GPT-4o
        response = self.client.chat.completions.create(
            model="gpt-4o-mini", messages=self.messages
        )
        answer = response.choices[0].message.content
        self.messages.append({"role": "assistant", "content": answer})

        # Store the question in memory
        self.memory.add(question, user_id=user_id)
        return answer

    def get_memories(self, user_id):
        memories = self.memory.get_all(user_id=user_id)
        return [m["memory"] for m in memories["memories"]]

    def search_memories(self, query, user_id):
        memories = self.memory.search(query, user_id=user_id)
        return [m["memory"] for m in memories["memories"]]

In [16]:
# Usage example
user_id = "tam-1c"
ai_assistant = PersonalTravelAssistant()

In [17]:
def ask_travel_assistant():
    while True:
        question = input("Question: ")
        if question.lower() in ["q", "exit"]:
            print("Exiting...")
            break

        answer = ai_assistant.ask_question(question, user_id=user_id)
        print(f"Answer: {answer}")
        memories = ai_assistant.get_memories(user_id=user_id)
        print("Memories:")
        for memory in memories:
            print(f"- {memory}")
        print("-----")

In [18]:
ask_travel_assistant()

Answer: That sounds exciting! China is a country rich in history, culture, and diverse landscapes. Here are some things to consider when planning your trip:

1. **Destinations**: Popular cities include Beijing (for the Great Wall, Forbidden City), Shanghai (modern skyline, Yu Garden), Xi'an (Terracotta Army), and Chengdu (pandas). Depending on your interests, you might also consider visiting rural areas like Zhangjiajie or the picturesque Li River in Yangshuo.

2. **Visa Requirements**: Check the visa requirements for your nationality. Most travelers will need a visa to enter China, which should be applied for in advance.

3. **Best Time to Visit**: The best times to visit China are generally during spring (March to May) and autumn (September to November) when the weather is mild and pleasant.

4. **Language**: While major cities have some English-speaking residents, it’s helpful to learn a few basic Mandarin phrases. Apps like Google Translate can assist with communication.

5. **Cult